In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import networkx as nx
import warnings
import pickle
warnings.simplefilter('ignore') # for networkx drawing
import matplotlib.pylab
%matplotlib inline
matplotlib.pylab.rcParams["figure.figsize"] = (14, 18)

In [ ]:
datadir = pathlib.Path("/home/david/pro/scc/data")

In [ ]:
%run utils.py
#cache = ApprovedMatches(datadir / "approved.pickle")

# Fix the birthplace field

In [ ]:
changes = set()
for fn in sorted(datadir.glob("lc_*.csv")):
    year = int(re.search(r"\d{4}", fn.name).group(0))
    print(year)
    lines = []
    with fn.open("r", encoding="UTF-8") as fd:
        lines.append(next(fd))
        for line in fd:
            row = line.strip().split("|")
            # prøv at fikse fødesteder
            if "her i sognet" in row[5] or "heri sognet" in row[5] or row[5].replace(".", "") in ("i sognet", "her i s"):        
                # det kan være, der står noget om placering i parentes; brug det
                match1 = re.search(r"(her ?)?i sognet (\{|\[|\()(?P<place>.*?)(\]|\)|\})", row[5])
                # det kan også være, der står noget efter et komma
                match2 = re.search(r"(her ?)? i sognet, (<P<place>.*)", row[5])
                if match1:
                    new = match1.group("place")
                elif match2:
                    new = match2.group("place")
                else:
                    new = row[2]
                changes.add((row[5], new))
                row[5] = new
            lines.append("|".join(row) + "\n")
    with fn.open("w", encoding="UTF-8") as fd:
        fd.writelines(lines)

In [ ]:
re.search(r"(her ?)?i sognet (\[|\()(?P<place>.*?)(\]|\))", "her i sognet [tommerup)").group("place")

In [43]:
# for 1845 som eksempel
sorted((a,b) for (a,b) in changes if a not in ("her i sognet", "heri sognet", "i sognet"))

[('aabølling, her i sognet', 'føvling'),
 ('aagerup, her i sognet [vigersted], sorø', 'vigersted'),
 ('aalehuset ved nyborg, her i sognet [aunslev]', 'aunslev'),
 ('aalehuset, her i sognet [aunslev]', 'aunslev'),
 ('aalstrup her i sognet [falling]', 'falling'),
 ('aas her i sognet [skjoldborg]', 'skjoldborg'),
 ('alle her i sognet', 'damsholte'),
 ('alle her i sognet', 'folding'),
 ('alle her i sognet [stavning]', 'stavning'),
 ('alle heri sognet', 'folding'),
 ('alsbjerg, her i sognet', 'torslev'),
 ('alsted sogn her i sognet', 'erslev'),
 ('alstrup her i sognet', 'mariager landsogn'),
 ('alstrup her i sognet', 'strandby'),
 ('ammelhede, her i sognet', 'virring'),
 ('anelund her i sognet', 'gammel rye'),
 ('arnøje, her i sognet', 'hellested'),
 ('asaa her i sognet', 'dronninglund'),
 ('asaa heri sognet', 'dronninglund'),
 ('asbækholt heri sognet', 'dronninglund'),
 ('askildrup her i sognet randers amt', 'Ølst'),
 ('assens sogn ("her i sognet" er overstreget)', 'barrit'),
 ('attrup, he